In [2]:
import mmcv
import numpy as np
import os
import open3d as o3d
from open3d.web_visualizer import draw
from mmengine import load
import cv2

from mmdet3d.visualization import Det3DLocalVisualizer
from mmdet3d.structures import CameraInstance3DBoxes

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.
[Open3D INFO] Resetting default logger to print to terminal.


# DAIR BIN

In [3]:
bin_path = 'DAIR/000000/000000.bin'
# Read pc from bin file
# Read pc from bin file
with open(bin_path, 'rb') as f:
    points = np.fromfile(f, dtype=np.float32, count=-1).reshape([-1, 4])

pcd_bin = o3d.geometry.PointCloud()
pcd_bin.points = o3d.utility.Vector3dVector(points[:, :3])
print(pcd_bin)


PointCloud with 56285 points.


In [4]:
def get_lidar_in_cam_T():
    Tr_velo_to_cam = np.array([-0.032055018882740136, -0.9974518923884874, 0.020551248965447914, -2.190444561668236, 
                               -0.2240930139414797, 0.002986041494130043, -0.8756800120708629, 5.6360862566491905, 
                               0.9737455440255373, -0.041678350017788, -0.2023375046787095, 1.4163664770754851])
    Tr_velo_to_cam = Tr_velo_to_cam.reshape(3, 4)
    R = Tr_velo_to_cam[0:3, 0:3]
    T = Tr_velo_to_cam[0:3, 3]
    lidar_in_cam_T = np.eye(4)
    lidar_in_cam_T[0:3, 0:3] = R.T
    lidar_in_cam_T[0:3, 3] = -R.T @ T
    return lidar_in_cam_T

# Define function to generate 3D box from label in lidar coordinates
def generate_box(label):
    fields = label.split(' ')
    obj_type = fields[0]
    dimensions = [float(fields[i]) for i in range(8, 11)]
    position = [float(fields[i]) for i in range(11, 14)]
    rotation_y = float(fields[14])

    # Create 3D box mesh
    box = o3d.geometry.TriangleMesh.create_box(width=dimensions[1], height=dimensions[2], depth=dimensions[0])
    box.paint_uniform_color([1.0, 0.0, 0.0]) # Set color to red

    position[0] = position[0] + dimensions[2]/2
    position[1] = position[1] - dimensions[0]/2
    position[2] = position[2] - dimensions[1]/2

    # Rotate box
    rotation = box.get_rotation_matrix_from_xyz((0, 0, rotation_y))
    box.rotate(rotation)

    # Convert position to lidar coordinates
    lidar_in_cam_T = get_lidar_in_cam_T()
    pos_in_cam = np.array(position + [1])
    pos_in_lidar = lidar_in_cam_T @ pos_in_cam
    box.translate([pos_in_lidar[0], pos_in_lidar[1], pos_in_lidar[2]])
    
    # Draw rectangle in image
    image = cv2.imread('DAIR/000000/000000.png')
    xmin, xmax, ymin, ymax = int(float(fields[4])), int(float(fields[6])), int(float(fields[5])), int(float(fields[7]))
    cv2.rectangle(image, (xmin, ymin), (xmax, ymax), (0, 255, 0), 2)  # Green color, thickness = 2
    # Save the image with the rectangle
    cv2.imwrite('DAIR/000000/000000.png', image)  # Replace 'image_with_rectangle.jpg' with the desired file name and format

    return box

label1 = "Cyclist 1 0 2.734332467096819 1528.72644 289.33963 1597.9943839999999 365.06311 1.762347 1.649016 0.761443 15.45390690610019 -4.743863383912665 54.198635350304585 -1.6607597119606952"
label2 = 'Car 0 0 4.670307121793625 960.894714 397.073822 1069.800171 499.614593 1.603673 4.216247 1.892322 1.3023832310605297 -1.5326935470470806 39.53224734353478 0.023192134753643994'
label3 = 'Car 0 3 4.335286106350193 1433.344849 1010.11145 1920.0 1080.0 1.505857 4.240506 1.826533 4.473331511422382 4.2188503292016195 14.534916755780703 -0.04451658908474994'
label4 = 'Car 1 3 1.9535737262218003 0 228.458496 56.103931 287.786346 1.427104 4.289026 1.808059 -30.12074073506167 -8.65311139425015 69.5201241021645 -3.134923340425329'
box1 = generate_box(label1)
box2 = generate_box(label2)
box3 = generate_box(label3)
box4 = generate_box(label4)

# Visualize box mesh
draw([pcd_bin, box1, box2, box3, box4], width=900, height=600, point_size=2)

[1.762347, 1.649016, 0.761443]
[1.603673, 4.216247, 1.892322]
[1.505857, 4.240506, 1.826533]
[1.427104, 4.289026, 1.808059]
[Open3D INFO] Window window_0 created.
[Open3D INFO] EGL headless mode enabled.
[Open3D INFO] ICE servers: ["stun:stun.l.google.com:19302", "turn:user:password@34.69.27.100:3478", "turn:user:password@34.69.27.100:3478?transport=tcp"]
FEngine (64 bits) created at 0x7f979f2f6010 (threading is enabled)
[Open3D INFO] Set WEBRTC_STUN_SERVER environment variable add a customized WebRTC STUN server.
[Open3D INFO] WebRTC Jupyter handshake mode enabled.
EGL(1.5)
OpenGL(4.1)


WebVisualizer(window_uid='window_0')

[Open3D INFO] [Called HTTP API (custom handshake)] /api/getIceServers
[Open3D INFO] [Called HTTP API (custom handshake)] /api/call
[Open3D INFO] [Called HTTP API (custom handshake)] /api/addIceCandidate
[Open3D INFO] [Called HTTP API (custom handshake)] /api/addIceCandidate
[Open3D INFO] [Called HTTP API (custom handshake)] /api/addIceCandidate
[Open3D INFO] [Called HTTP API (custom handshake)] /api/addIceCandidate
[Open3D INFO] [Called HTTP API (custom handshake)] /api/addIceCandidate
[Open3D INFO] [Called HTTP API (custom handshake)] /api/addIceCandidate
[Open3D INFO] [Called HTTP API (custom handshake)] /api/getIceCandidate
[Open3D INFO] [Called HTTP API (custom handshake)] /api/addIceCandidate
[Open3D INFO] DataChannelObserver::OnStateChange label: ServerDataChannel, state: open, peerid: 0.33897186708787275
[Open3D INFO] DataChannelObserver::OnStateChange label: ClientDataChannel, state: open, peerid: 0.33897186708787275
[Open3D INFO] Sending init frames to window_0.


[000:000][437983] (stun_port.cc:96): Binding request timed out from 163.117.150.x:35349 (enp4s0)


## Example 2: 04860

In [5]:
bin_path = 'DAIR/004860/004860.bin'
# Read pc from bin file
# Read pc from bin file
with open(bin_path, 'rb') as f:
    points = np.fromfile(f, dtype=np.float32, count=-1).reshape([-1, 4])

pcd_bin = o3d.geometry.PointCloud()
pcd_bin.points = o3d.utility.Vector3dVector(points[:, :3])
print(pcd_bin)


PointCloud with 54365 points.


In [15]:
def get_lidar_in_cam_T():
    Tr_velo_to_cam = np.array([-0.03030349774397828, -1.00458142729985, -0.014115943716385967, -2.256601052723574,
                               -0.22437570965542747, 0.0035552292126834464, -0.8538802187818876, 5.633608625376564,
                                0.9740701880036601, -0.04242754550762507, -0.203730541714936, 1.9350541378597665])
    
    Tr_velo_to_cam = Tr_velo_to_cam.reshape(3, 4)
    R = Tr_velo_to_cam[0:3, 0:3]
    T = Tr_velo_to_cam[0:3, 3]
    lidar_in_cam_T = np.eye(4)
    lidar_in_cam_T[0:3, 0:3] = R.T
    lidar_in_cam_T[0:3, 3] = -R.T @ T
    return lidar_in_cam_T

# Define function to generate 3D box from label in lidar coordinates
def generate_box(label):
    fields = label.split(' ')
    obj_type = fields[0]
    dimensions = [float(fields[i]) for i in range(8, 11)]
    position = [float(fields[i]) for i in range(11, 14)]
    rotation_y = float(fields[14])

    # Create 3D box mesh
    box = o3d.geometry.TriangleMesh.create_box(width=dimensions[1], height=dimensions[2], depth=dimensions[0])
    box.paint_uniform_color([1.0, 0.0, 0.0]) # Set color to red

    position[0] = position[0] + dimensions[2]/2
    position[1] = position[1] - dimensions[0]/2
    position[2] = position[2] - dimensions[1]/2

    # Rotate box
    rotation = box.get_rotation_matrix_from_xyz((0, 0, rotation_y))
    box.rotate(rotation)
    
    # Convert position to lidar coordinates
    lidar_in_cam_T = get_lidar_in_cam_T()
    pos_in_cam = np.array(position + [1])
    pos_in_lidar = lidar_in_cam_T @ pos_in_cam
    box.translate([pos_in_lidar[0], pos_in_lidar[1], pos_in_lidar[2]])
    
    # Draw rectangle in image
    image = cv2.imread('DAIR/004860/004860.png')
    xmin, xmax, ymin, ymax = int(float(fields[4])), int(float(fields[6])), int(float(fields[5])), int(float(fields[7]))
    cv2.rectangle(image, (xmin, ymin), (xmax, ymax), (0, 255, 0), 2)  # Green color, thickness = 2
    # Save the image with the rectangle
    cv2.imwrite('DAIR/004860/004860.png', image) 

    return box

label1 = "Car 0 0 1.735128339521593 441.219452 192.290527 514.722656 248.033279 1.722938 4.250462 2.009179 -17.303386034031742 -11.41159629331521 82.0519167700954 3.1295692685401018"
label2 = 'Car 0 0 1.9292440979882186 64.748047 265.136322 193.25518799999998 367.75943 1.718504 4.745133 2.009396 -19.988120037397977 -4.975312822193446 54.15725924916455 -3.106701309344368'
label3 = 'Car 0 3 1.9748159748851488 0 273.444153 61.432224 367.008179 1.751429 4.631501 2.057613 -23.924812451064863 -4.999316794464347 54.42000697655673 -3.1213094572278357'
label4 = 'Car 1 0 1.8681144080847403 215.876404 244.711472 309.622498 308.089447 1.532154 4.263748 1.920395 -20.466887910439464 -7.529329681831728 65.5094273992565 -3.1167711506613345'
box1 = generate_box(label1)
box2 = generate_box(label2)
box3 = generate_box(label3)
box4 = generate_box(label4)

# Visualize box mesh
draw([pcd_bin, box1, box2, box3, box4], width=900, height=600, point_size=2)

[Open3D INFO] Window window_3 created.


WebVisualizer(window_uid='window_3')

[Open3D INFO] [Called HTTP API (custom handshake)] /api/getIceServers
[Open3D INFO] [Called HTTP API (custom handshake)] /api/call
[Open3D INFO] [Called HTTP API (custom handshake)] /api/addIceCandidate
[Open3D INFO] [Called HTTP API (custom handshake)] /api/addIceCandidate
[Open3D INFO] [Called HTTP API (custom handshake)] /api/addIceCandidate
[Open3D INFO] [Called HTTP API (custom handshake)] /api/addIceCandidate
[Open3D INFO] [Called HTTP API (custom handshake)] /api/addIceCandidate
[Open3D INFO] [Called HTTP API (custom handshake)] /api/addIceCandidate
[Open3D INFO] [Called HTTP API (custom handshake)] /api/getIceCandidate
[Open3D INFO] [Called HTTP API (custom handshake)] /api/addIceCandidate
[Open3D INFO] DataChannelObserver::OnStateChange label: ServerDataChannel, state: open, peerid: 0.8308912464998413
[Open3D INFO] DataChannelObserver::OnStateChange label: ClientDataChannel, state: open, peerid: 0.8308912464998413
[Open3D INFO] Sending init frames to window_3.


[2048:276][437983] (stun_port.cc:96): Binding request timed out from 163.117.150.x:35964 (enp4s0)


## Example 3: 10000

In [7]:
bin_path = 'DAIR/010000/010000.bin'
# Read pc from bin file
# Read pc from bin file
with open(bin_path, 'rb') as f:
    points = np.fromfile(f, dtype=np.float32, count=-1).reshape([-1, 4])

pcd_bin = o3d.geometry.PointCloud()
pcd_bin.points = o3d.utility.Vector3dVector(points[:, :3])
print(pcd_bin)

PointCloud with 52907 points.


[502:073][437983] (stun_port.cc:96): Binding request timed out from 163.117.150.x:46210 (enp4s0)


In [8]:
def get_lidar_in_cam_T():
    Tr_velo_to_cam = np.array([-0.032055018882740136, -0.9974518923884874, 0.020551248965447914, -2.190444561668236, 
                               -0.2240930139414797, 0.002986041494130043, -0.8756800120708629, 5.6360862566491905, 
                               0.9737455440255373, -0.041678350017788, -0.2023375046787095, 1.4163664770754851])
    Tr_velo_to_cam = np.array([0.007438236214504291, -0.9987793825345748, 0.0033973101819174296, -2.654400296356563,
                               -0.18545815331876273, -0.0016061225554877922, -0.8486371688204063, 6.1008711543605605,
                                0.9828008692703378, 0.012785829992919598, -0.15783375715452574, 0.9742049179614991])
    
    Tr_velo_to_cam = Tr_velo_to_cam.reshape(3, 4)
    R = Tr_velo_to_cam[0:3, 0:3]
    T = Tr_velo_to_cam[0:3, 3]
    lidar_in_cam_T = np.eye(4)
    lidar_in_cam_T[0:3, 0:3] = R.T
    lidar_in_cam_T[0:3, 3] = -R.T @ T
    return lidar_in_cam_T

# Define function to generate 3D box from label in lidar coordinates
def generate_box(label):
    fields = label.split(' ')
    obj_type = fields[0]
    dimensions = [float(fields[i]) for i in range(8, 11)]
    position = [float(fields[i]) for i in range(11, 14)]
    rotation_y = float(fields[14])

    # Create 3D box mesh
    box = o3d.geometry.TriangleMesh.create_box(width=dimensions[1], height=dimensions[2], depth=dimensions[0])
    box.paint_uniform_color([1.0, 0.0, 0.0]) # Set color to red

    position[0] = position[0] + dimensions[2]/2
    position[1] = position[1] - dimensions[0]/2
    position[2] = position[2] - dimensions[1]/2

    # Rotate box
    rotation = box.get_rotation_matrix_from_xyz((0, 0, rotation_y))
    box.rotate(rotation)

    # Convert position to lidar coordinates
    lidar_in_cam_T = get_lidar_in_cam_T()
    pos_in_cam = np.array(position + [1])
    pos_in_lidar = lidar_in_cam_T @ pos_in_cam
    box.translate([pos_in_lidar[0], pos_in_lidar[1], pos_in_lidar[2]])
    
    # Draw rectangle in image
    image = cv2.imread('DAIR/010000/010000.png')
    xmin, xmax, ymin, ymax = int(float(fields[4])), int(float(fields[6])), int(float(fields[5])), int(float(fields[7]))
    cv2.rectangle(image, (xmin, ymin), (xmax, ymax), (0, 255, 0), 2)  # Green color, thickness = 2
    # Save the image with the rectangle
    cv2.imwrite('DAIR/010000/010000.png', image)  # Replace 'image_with_rectangle.jpg' with the desired file name and format

    return box

label1 = "Car 0 0 1.6462837042110265 547.049988 271.122223 630.6881109999999 337.942719 1.526688 4.371019 1.818874 -11.297903050171037 -6.201772262967185 75.48462916603195 3.0621975127851484"
label2 = 'Car 0 0 1.6812379415239458 715.357117 134.764725 744.769837 159.246002 1.423728 4.283895 1.932858 -18.118293478821244 -26.972533269459298 188.66372792352476 -3.134575179617945'
label3 = 'Car 0 0 4.563342955372911 1136.35083 442.261139 1281.718384 564.741699 1.574384 4.565282 1.93222 3.3154922575600576 -0.45785586759627517 43.56110840936029 -0.07939498324800333'
label4 = 'Car 0 0 -0.17045890666725522 845.8255 300.374969 1033.715942 369.713531 1.46991 4.500235 1.955691 -1.2690746216172615 -5.117270224809096 68.37306859456952 1.365986867102916'
box1 = generate_box(label1)
box2 = generate_box(label2)
box3 = generate_box(label3)
box4 = generate_box(label4)

# Visualize box mesh
draw([pcd_bin, box1, box2, box3, box4], width=900, height=600, point_size=2)

[Open3D INFO] Window window_2 created.


WebVisualizer(window_uid='window_2')

[Open3D INFO] [Called HTTP API (custom handshake)] /api/getIceServers
[Open3D INFO] [Called HTTP API (custom handshake)] /api/call
[Open3D INFO] [Called HTTP API (custom handshake)] /api/addIceCandidate
[Open3D INFO] [Called HTTP API (custom handshake)] /api/addIceCandidate
[Open3D INFO] [Called HTTP API (custom handshake)] /api/addIceCandidate
[Open3D INFO] [Called HTTP API (custom handshake)] /api/addIceCandidate
[Open3D INFO] [Called HTTP API (custom handshake)] /api/addIceCandidate
[Open3D INFO] [Called HTTP API (custom handshake)] /api/addIceCandidate
[Open3D INFO] [Called HTTP API (custom handshake)] /api/addIceCandidate
[Open3D INFO] [Called HTTP API (custom handshake)] /api/addIceCandidate
[Open3D INFO] [Called HTTP API (custom handshake)] /api/addIceCandidate
[Open3D INFO] [Called HTTP API (custom handshake)] /api/getIceCandidate
[Open3D INFO] DataChannelObserver::OnStateChange label: ServerDataChannel, state: open, peerid: 0.6271070798029654
[Open3D INFO] DataChannelObserver:

[555:900][437983] (stun_port.cc:96): Binding request timed out from 163.117.150.x:56339 (enp4s0)
